# Code for Protine stablity predection

## Common factors that affect the stability are temperature, light, pH, oxidation and enzymatic degradation.

In [120]:
# Import all the requried modules

## EDA of the source data

## import all the libs

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import biopandas as bpd
from biopandas.pdb import PandasPdb
import pandas_profiling
from IPython.core.display import display, HTML

# visualization

import plotly.express as px
import plotly.graph_objects as go

# Import data files

In [121]:
## import data sets and updates

df_sample_sub = pd.read_csv('sample_submission.csv')
df_test=pd.read_csv('test.csv')
df_train=pd.read_csv('train.csv')
df_train_upd=pd.read_csv('train_updates_20220929.csv')


ppdb = PandasPdb()
df_stprd = ppdb.read_pdb('wildtype_structure_prediction_af2.pdb')



In [122]:

#update data set
df_train_upd = pd.read_csv("train_updates_20220929.csv", index_col="seq_id")

all_features_nan = df_train_updates.isnull().all("columns")
drop_indices = df_train_upd[all_features_nan].index
df_train = df_train.drop(index=drop_indices)

swap_ph_tm_indx = df_train_upd[~all_features_nan].index
df_train.loc[swap_ph_tm_indx, ["pH", "tm"]] = df_train_upd.loc[swap_ph_tm_indx, ["pH", "tm"]]


In [123]:
# get leng

df_train['protein_sequence']= df_train['protein_sequence'].astype(str)

df_train["protein_sequence_len"] = df_train["protein_sequence"].apply(lambda x: len(x))
df_test["protein_sequence_len"] = df_test["protein_sequence"].apply(lambda x: len(x))

In [124]:
df_train.tail()

,seq_id,protein_sequence,pH,data_source,tm,protein_sequence_len
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8,549
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2,469
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6,128
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7,593
31389,31389,YYVPDEYWQSLEVAHKLTFGYGYLTWEWVQGIRSYVYPLLIAGLYK...,7.0,doi.org/10.1038/s41592-020-0801-4,37.6,537


In [125]:

from scipy.sparse import csr_matrix

df_train = df_train[df_train["protein_sequence_len"]<=221]
df_train.reset_index(inplace=True)
sequences = [list(string) for string in df_train["protein_sequence"].values.tolist()]
sequences_train = pd.DataFrame(sequences)
sequences_train.head()

,0,1,2,3,4,5,6,7,8,9,...,211,212,213,214,215,216,217,218,219,220
0,A,A,F,Q,V,T,S,N,E,I,...,None,None,None,None,None,None,None,None,None,None
1,A,A,G,G,Q,P,Q,G,A,T,...,A,Q,Q,Q,C,N,None,None,None,None
2,A,A,I,G,I,G,I,L,G,G,...,None,None,None,None,None,None,None,None,None,None
3,A,A,K,S,G,D,A,E,E,A,...,None,None,None,None,None,None,None,None,None,None
4,A,A,L,A,L,G,L,P,A,F,...,None,None,None,None,None,None,None,None,None,None


In [126]:
from sklearn.preprocessing import LabelEncoder

sequences_train = sequences_train.apply(LabelEncoder().fit_transform)
sequences_train["tm"] = df_train["tm"]
sequences_train.head()

,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,tm
0,0,0,4,13,17,16,15,11,3,7,...,20,19,20,20,20,20,20,20,18,49.7
1,0,0,5,5,13,12,13,5,0,16,...,13,13,13,1,11,20,20,20,18,45.1
2,0,0,7,5,7,5,7,9,5,5,...,20,19,20,20,20,20,20,20,18,62.8
3,0,0,8,15,5,2,0,3,3,0,...,20,19,20,20,20,20,20,20,18,36.3
4,0,0,9,0,9,5,9,12,0,4,...,20,19,20,20,20,20,20,20,18,83.0


In [149]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X = sequences_train.loc[:, sequences_train.columns != "tm"]
y = sequences_train.loc[:, sequences_train.columns == "tm"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
# create an xgboost regression model
model = xgb.XGBRegressor(n_estimators=500, max_depth=15)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [150]:
from scipy import stats

stats.spearmanr(y_test, y_pred)
#stats.pearsonr(y_test, y_pred)

SpearmanrResult(correlation=0.271193233173363, pvalue=3.5430220226940706e-79)

In [129]:
from scipy.sparse import csr_matrix

df_test = df_test[df_test["protein_sequence_len"]<=221]
sequences = [list(string) for string in df_test["protein_sequence"].values.tolist()]
sequences_test = pd.DataFrame(sequences)
sequences_test = sequences_test.apply(LabelEncoder().fit_transform)
sequences_test.head()

,0,1,2,3,4,5,6,7,8,9,...,211,212,213,214,215,216,217,218,219,220
0,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6
1,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6
2,0,0,0,0,0,0,0,0,0,0,...,10,11,6,2,5,16,11,4,4,13
3,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6
4,0,0,0,0,0,0,0,0,0,0,...,7,11,7,5,1,8,13,15,2,6


In [130]:
submission = pd.DataFrame()
submission["tm"] = model.predict(sequences_test)
submission["seq_id"] = df_test["seq_id"]
submission.to_csv("submission.csv", index=False)